# Bank Marketing Dataset-UCI Machine learning reporsitory

In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
from sklearn import preprocessing
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

In [9]:
data=pd.read_csv('bank-additional-full.csv',sep=';')

In [10]:
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

### Pre-processing 1

Label encoding of Education and One-hot encoding of categorical fields

In [4]:
class Feature_engineering(BaseEstimator, TransformerMixin):
    def __init__(self):
        return

    def fit(self, X=data, y=None):
        data=X.copy()
        data.education=preprocessing.LabelEncoder().fit_transform(data.education)
        cols=[]
        for i in data.columns:
            if(data[i].dtype=='O'):
                cols.append(i)
        data_dummies=pd.get_dummies(data,prefix=cols,columns=cols)
        self.data_columns=data_dummies.columns
        return self
        

    
    def transform(self,X=data):
        data=X.copy()
        data.education=preprocessing.LabelEncoder().fit_transform(data.education)
        cols=[]
        for i in data.columns:
            if(data[i].dtype=='O'):
                cols.append(i)
        data_dummies=pd.get_dummies(data,prefix=cols,columns=cols)
        data_dummies=data_dummies.reindex(columns = self.data_columns, fill_value=0)
        data=data.drop(cols,axis=1)
        x = pd.concat([data, data_dummies], axis=1)
        x=x.loc[:,~x.columns.duplicated()]
        return x
    def transform_y(data):
        y_dict={
            'yes':1,
            'no':0
        }
        data=data.map(y_dict)
        return data

### Pre-processing 2


Ordinal encoding of Education and One-hot encoding of categorical fields

In [6]:
class Feature_engineering(BaseEstimator, TransformerMixin):
    def __init__(self):
        return

    def fit(self, X=data, y=None):
        data=X.copy()
        edu_dict={
        'unknown':0,
        'illiterate':1,

        'basic.4y':2,
        'basic.6y':3,
        'basic.9y':4, 
        'high.school':5, 
           'professional.course':6,
        'university.degree':7
        }
        data['education']=data['education'].map(edu_dict)
        cols=[]
        cols=[]
        for i in data.columns:
            if(data[i].dtype=='O'):
                cols.append(i)
        data_dummies=pd.get_dummies(data,prefix=cols,columns=cols)
        self.data_columns=data_dummies.columns
        return self
        

    
    def transform(self,X=data):
        data=X.copy()
        edu_dict={
        'unknown':0,
        'basic.4y':1,
        'basic.6y':2,
        'basic.9y':3, 
        'high.school':4, 
        'illiterate':5,
           'professional.course':6,
        'university.degree':7
        }
        data['education']=data['education'].map(edu_dict)
        cols=[]
        for i in data.columns:
            if(data[i].dtype=='O'):
                cols.append(i)
        data_dummies=pd.get_dummies(data,prefix=cols,columns=cols)
        data_dummies=data_dummies.reindex(columns = self.data_columns, fill_value=0)
        data=data.drop(cols,axis=1)
        x = pd.concat([data, data_dummies], axis=1)
        x=x.loc[:,~x.columns.duplicated()]
        return x
    def transform_y(data):
        y_dict={
            'yes':1,
            'no':0
        }
        data=data.map(y_dict)
        return data

### Pipeline and Strarified K fold Cross-validation on Decision Tree

In [6]:
X=data.iloc[:,:-1]
y=Feature_engineering.transform_y(data.iloc[:,-1])

In [7]:
cv=StratifiedKFold(n_splits=10, random_state=0, shuffle=True)
pipeline = Pipeline([('Pre-processing',Feature_engineering()),('Tree',DecisionTreeClassifier(random_state=0,max_depth=2))])
scores = cross_validate(pipeline,X,y,cv=cv,scoring='roc_auc',return_train_score=True)
print("Mean training ROC AUC score: ",np.mean(scores['train_score'])," and mean testing ROC AUC score: ",np.mean(scores['test_score']))

Mean training ROC AUC score:  0.8521955540028777  and mean testing ROC AUC score:  0.8499926079714853


In [14]:
scores

{'fit_time': array([2.79338431, 2.68430042, 2.96471882, 2.8274498 , 2.72836328,
        2.88061118, 2.65626407, 2.64867353, 2.76072836, 2.67397094]),
 'score_time': array([0.09555292, 0.08388901, 0.06148648, 0.08757496, 0.06433558,
        0.08884931, 0.12814283, 0.05792904, 0.09067822, 0.08883286]),
 'estimator': [Pipeline(steps=[('Pre-processing', Feature_engineering()),
                  ('forest',
                   RandomForestClassifier(n_estimators=50, random_state=42))]),
  Pipeline(steps=[('Pre-processing', Feature_engineering()),
                  ('forest',
                   RandomForestClassifier(n_estimators=50, random_state=42))]),
  Pipeline(steps=[('Pre-processing', Feature_engineering()),
                  ('forest',
                   RandomForestClassifier(n_estimators=50, random_state=42))]),
  Pipeline(steps=[('Pre-processing', Feature_engineering()),
                  ('forest',
                   RandomForestClassifier(n_estimators=50, random_state=42))]),
  Pip

### Pipeline and Strarified K fold Cross-validation on Random Forest


Enter those fields to be retained based on feature importance

In [5]:
data=pd.read_csv('bank-additional-full.csv',sep=';')
y=data.iloc[:,-1]
data=data.loc[:,['education','duration','euribor3m','campaign','nr.employed']]
data['y']=y

In [6]:
X=data.iloc[:,:-1]
y=Feature_engineering.transform_y(data.iloc[:,-1])
cv=StratifiedKFold(n_splits=10, random_state=0, shuffle=True)
pipeline = Pipeline([('Pre-processing',Feature_engineering()),('forest',RandomForestClassifier(n_estimators=50,random_state=42))])
scores = cross_validate(pipeline,X,y,cv=cv,scoring='roc_auc',return_train_score=True,return_estimator =True)
print("Mean training ROC AUC score: ",np.mean(scores['train_score'])," and mean testing ROC AUC score: ",np.mean(scores['test_score']))

Mean training ROC AUC score:  0.999977659756567  and mean testing ROC AUC score:  0.9274717370725447


#### Generating the feature importances matrix

In [7]:
feature_importances=pd.DataFrame()
for idx,estimator in enumerate(scores['estimator']):
    ft_estimator=pd.DataFrame({
        'columns order for estimator_{}'.format(idx):Feature_engineering().fit_transform(X).columns,
        'importance_estimator_{}'.format(idx):estimator.steps[1][1].feature_importances_}).sort_values(['importance_estimator_{}'.format(idx)], ascending=False)
    ft_estimator.reset_index(inplace=True,drop=True)
    feature_importances=pd.concat([feature_importances,ft_estimator],axis=1)

In [8]:
feature_importances.head()

,columns order for estimator_0,importance_estimator_0,columns order for estimator_1,importance_estimator_1,columns order for estimator_2,importance_estimator_2,columns order for estimator_3,importance_estimator_3,columns order for estimator_4,importance_estimator_4,columns order for estimator_5,importance_estimator_5,columns order for estimator_6,importance_estimator_6,columns order for estimator_7,importance_estimator_7,columns order for estimator_8,importance_estimator_8,columns order for estimator_9,importance_estimator_9
0,duration,0.502982,duration,0.500167,duration,0.498759,duration,0.509208,duration,0.505834,duration,0.504940,duration,0.503919,duration,0.504191,duration,0.506110,duration,0.495486
1,euribor3m,0.231585,euribor3m,0.236763,euribor3m,0.236971,euribor3m,0.232916,euribor3m,0.237678,euribor3m,0.234511,euribor3m,0.232490,euribor3m,0.231638,euribor3m,0.228175,euribor3m,0.236730
2,nr.employed,0.131849,nr.employed,0.129056,nr.employed,0.129952,nr.employed,0.124886,nr.employed,0.124644,nr.employed,0.127439,nr.employed,0.130281,nr.employed,0.130895,nr.employed,0.131329,nr.employed,0.134192
3,education,0.069361,education,0.069825,education,0.070282,education,0.069955,education,0.070170,education,0.069918,education,0.070638,education,0.069076,education,0.072033,education,0.069133
4,campaign,0.064223,campaign,0.064189,campaign,0.064036,campaign,0.063036,campaign,0.061674,campaign,0.063192,campaign,0.062671,campaign,0.064200,campaign,0.062352,campaign,0.064460


### Pipeline inclduing SMOTE and Undersampling on Random Forest


Enter those fields to be retained based on feature importance

In [22]:
data=pd.read_csv('bank-additional-full.csv',sep=';')
y=data.iloc[:,-1]
data=data.loc[:,['education','duration','euribor3m','campaign','nr.employed']]
data['y']=y

To deal with class imbalance , we oversample the minority class and undersampling the majority class

In [23]:
X=data.iloc[:,:-1]
y=Feature_engineering.transform_y(data.iloc[:,-1])

In [25]:
from imblearn.pipeline import Pipeline

over = SMOTE(sampling_strategy=0.2)
under = RandomUnderSampler(sampling_strategy=0.5)
pipeline = Pipeline([('Pre-processing',Feature_engineering()),('over', over),('under', under),('forest',RandomForestClassifier(n_estimators=50,random_state=42))])
pipeline = pipeline.fit(X,y)
scores = cross_validate(pipeline,X,y,cv=10,scoring='roc_auc',return_train_score=True,return_estimator =True)
print("Mean training ROC AUC score: ",np.mean(scores['train_score'])," and mean testing ROC AUC score: ",np.mean(scores['test_score']))

Mean training ROC AUC score:  0.9890644646015077  and mean testing ROC AUC score:  0.6054753740668902


#### Generating the feature importances matrix

In [26]:
feature_importances=pd.DataFrame()
for idx,estimator in enumerate(scores['estimator']):
    ft_estimator=pd.DataFrame({
        'columns order for estimator_{}'.format(idx):Feature_engineering().fit_transform(X).columns,
        'importance_estimator_{}'.format(idx):estimator.steps[3][1].feature_importances_}).sort_values(['importance_estimator_{}'.format(idx)], ascending=False)
    ft_estimator.reset_index(inplace=True,drop=True)
    feature_importances=pd.concat([feature_importances,ft_estimator],axis=1)

In [27]:
feature_importances.head()

,columns order for estimator_0,importance_estimator_0,columns order for estimator_1,importance_estimator_1,columns order for estimator_2,importance_estimator_2,columns order for estimator_3,importance_estimator_3,columns order for estimator_4,importance_estimator_4,columns order for estimator_5,importance_estimator_5,columns order for estimator_6,importance_estimator_6,columns order for estimator_7,importance_estimator_7,columns order for estimator_8,importance_estimator_8,columns order for estimator_9,importance_estimator_9
0,duration,0.438947,duration,0.443180,duration,0.462145,duration,0.481345,duration,0.447328,duration,0.484708,duration,0.497383,duration,0.471656,duration,0.485411,duration,0.420856
1,euribor3m,0.267995,euribor3m,0.253182,euribor3m,0.249284,euribor3m,0.229776,euribor3m,0.262762,euribor3m,0.299998,euribor3m,0.248136,euribor3m,0.290500,euribor3m,0.246097,euribor3m,0.256309
2,nr.employed,0.201433,nr.employed,0.209383,nr.employed,0.196248,nr.employed,0.201336,nr.employed,0.199023,nr.employed,0.120398,nr.employed,0.157669,nr.employed,0.154253,nr.employed,0.174260,nr.employed,0.248353
3,education,0.050704,education,0.052751,education,0.051803,education,0.047347,education,0.050733,education,0.053298,education,0.052102,education,0.044740,education,0.049292,education,0.040310
4,campaign,0.040922,campaign,0.041504,campaign,0.040520,campaign,0.040195,campaign,0.040154,campaign,0.041598,campaign,0.044709,campaign,0.038851,campaign,0.044939,campaign,0.034172
